In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import warnings

warnings.filterwarnings('ignore')
pd.set_option('display.max_columns', 100)


In [ ]:
# Use your local path or adjust if loading from data/ folder
df = pd.read_csv('../data/steam_games.csv', low_memory=False)

# Check shape and columns
print(f"Rows: {df.shape[0]}, Columns: {df.shape[1]}")
df.head()


In [ ]:
# Check nulls and datatypes
df.info()
df.isnull().sum().sort_values(ascending=False).head(20)


In [ ]:
# Describe numerical columns
df.describe()


In [ ]:
# Top frequent values in object columns
for col in df.select_dtypes(include='object').columns:
    print(f"\n{col} — Unique: {df[col].nunique()}")
    print(df[col].value_counts().head(5))


In [ ]:
plt.figure(figsize=(10, 5))
sns.histplot(df['price'], bins=50, kde=True)
plt.title('Distribution of Game Prices')
plt.xlabel('Price ($)')
plt.ylabel('Frequency')
plt.show()


In [ ]:
missing = df.isnull().mean() * 100
missing[missing > 0].sort_values(ascending=False)
